In [ ]:
import sqlite3
import os
import pandas as pd

DB_PATH = os.path.join("..", "database", "air_tracker.db")
conn = sqlite3.connect(DB_PATH)

print("Connected to:", os.path.abspath(DB_PATH))

Connected to: /Users/jyoti/AIML_Projects/air_tracker/database/air_tracker.db


In [37]:
pd.read_sql("""
SELECT name
FROM pragma_table_info('airport');
""", conn)

,name
0,airport_id
1,icao_code
2,iata_code
3,name
4,city
5,country
6,continent
7,latitude
8,longitude
9,timezone


In [33]:
pd.read_sql("""
SELECT name
FROM pragma_table_info('aircraft');
""", conn)

,name
0,aircraft_id
1,registration
2,model
3,manufacturer
4,icao_type_code
5,owner


In [34]:
pd.read_sql("""
SELECT name
FROM pragma_table_info('airport_delays');
""", conn)

,name
0,delay_id
1,airport_iata
2,delay_date
3,total_flights
4,delayed_flights
5,avg_delay_min
6,median_delay_min
7,canceled_flights


In [35]:
pd.read_sql("""
SELECT name
FROM pragma_table_info('flights');
""", conn)

,name
0,flight_number
1,airline_name
2,aircraft_registration
3,origin_iata
4,destination_iata
5,scheduled_time
6,actual_time
7,status
8,flight_type


### ✅ QUERY 1
Show the total number of flights for each aircraft model, listing the model and its count.

In [ ]:
query_1 = """
SELECT
    ac.model,
    COUNT(*) AS flight_count
FROM flights f
JOIN aircraft ac
    ON f.aircraft_registration = ac.registration
GROUP BY ac.model
ORDER BY flight_count DESC;
"""
pd.read_sql(query_1, conn)

,model,flight_count
0,A21N,210
1,A20N,199
2,B38M,91
3,A320,64
4,AT75,33
5,B738,29
6,None,17
7,B788,13
8,A321,10
9,B789,8


In [ ]:
query_2 = """
SELECT
    ac.registration,
    ac.model,
    COUNT(*) AS flight_count
FROM flights f
JOIN aircraft ac
    ON f.aircraft_registration = ac.registration
GROUP BY ac.registration, ac.model
HAVING COUNT(*) > 5
ORDER BY flight_count DESC;
"""
pd.read_sql(query_2, conn)

,registration,model,flight_count
0,VT-NCM,A21N,10
1,VT-NCO,A21N,10
2,VT-BOM,A320,9
3,VT-NHH,A21N,9
4,VT-IBF,A21N,8
5,VT-TNH,A20N,8
6,VT-BWH,B38M,7
7,VT-BWS,B38M,7
8,VT-IMW,A21N,7
9,VT-NCV,A21N,7


In [ ]:
query_3 = """
SELECT
    a.name AS airport_name,
    COUNT(*) AS outbound_flights
FROM flights f
JOIN airport a
    ON f.origin_iata = a.iata_code
WHERE f.flight_type = 'departure'
GROUP BY a.name
HAVING COUNT(*) > 5
ORDER BY outbound_flights DESC;
"""
pd.read_sql(query_3, conn)

,airport_name,outbound_flights
0,Bangalore Bengaluru,87
1,New Delhi Indira Gandhi,40
2,London Heathrow,36
3,Mumbai Chhatrapati Shivaji,32
4,New York John F Kennedy,27
5,Singapore Changi,23
6,Dubai,20
7,Paris Charles de Gaulle,19
8,Hyderabad Rajiv Gandhi,18
9,Chennai,18


In [12]:
query_4 = """
SELECT
    a.name AS airport_name,
    a.city,
    COUNT(*) AS arrival_count
FROM flights f
JOIN airport a
    ON f.destination_iata = a.iata_code
WHERE f.flight_type = 'arrival'
GROUP BY a.name, a.city
ORDER BY arrival_count DESC
LIMIT 3;
"""
pd.read_sql(query_4, conn)


,airport_name,city,arrival_count
0,Bangalore Bengaluru,Bangalore,80
1,New Delhi Indira Gandhi,New Delhi,63
2,Mumbai Chhatrapati Shivaji,Mumbai,36


In [ ]:
query_5 = """
SELECT
    f.flight_number,
    f.origin_iata,
    f.destination_iata,
    CASE
        WHEN o.country = d.country THEN 'Domestic'
        ELSE 'International'
    END AS flight_category
FROM flights f
LEFT JOIN airport o
    ON f.origin_iata = o.iata_code
LEFT JOIN airport d
    ON f.destination_iata = d.iata_code
WHERE o.country IS NOT NULL
  AND d.country IS NOT NULL;
"""
pd.read_sql(query_5, conn)

,flight_number,origin_iata,destination_iata,flight_category


In [ ]:
query_6 = """
SELECT
    f.flight_number,
    f.aircraft_registration,
    o.name AS departure_airport,
    f.scheduled_time AS arrival_time
FROM flights f
LEFT JOIN airport o
    ON f.origin_iata = o.iata_code
WHERE UPPER(f.destination_iata) = 'DEL'
  AND LOWER(f.flight_type) = 'arrival'
ORDER BY arrival_time DESC
LIMIT 5;
"""
pd.read_sql(query_6, conn)

,flight_number,aircraft_registration,departure_airport,arrival_time
0,AI 2807,VT-EXP,None,2026-01-02 17:30Z
1,6E 844,VT-NCV,None,2026-01-02 17:30Z
2,QP 1824,VT-YAU,None,2026-01-02 17:25Z
3,AI 2809,VT-TNB,None,2026-01-02 16:55Z
4,AI 2512,VT-TNH,None,2026-01-02 16:30Z


In [15]:
query_7 = """
SELECT
    a.name,
    a.city
FROM airport a
LEFT JOIN flights f
    ON a.iata_code = f.destination_iata
WHERE f.destination_iata IS NULL;
"""
pd.read_sql(query_7, conn)


,name,city


In [16]:
query_8 = """
SELECT
    airline_name,
    SUM(CASE WHEN status = 'On Time' THEN 1 ELSE 0 END) AS on_time,
    SUM(CASE WHEN status = 'Delayed' THEN 1 ELSE 0 END) AS delayed,
    SUM(CASE WHEN status = 'Cancelled' THEN 1 ELSE 0 END) AS cancelled
FROM flights
GROUP BY airline_name;
"""
pd.read_sql(query_8, conn)


,airline_name,on_time,delayed,cancelled
0,AHK Air Hong Kong Limited,0,0,0
1,AJet,0,0,0
2,AKJ,0,0,0
3,ANA,0,14,0
4,ASL France,0,0,0
...,...,...,...,...
198,airBaltic,0,1,0
199,easyJet,0,0,0
200,flyadeal,0,0,0
201,flydubai,0,0,0


In [ ]:
query_9 = """
SELECT
    f.flight_number,
    f.aircraft_registration,
    o.name AS origin_airport,
    d.name AS destination_airport,
    f.scheduled_time
FROM flights f
LEFT JOIN airport o
    ON f.origin_iata = o.iata_code
LEFT JOIN airport d
    ON f.destination_iata = d.iata_code
WHERE LOWER(f.status) IN ('cancelled', 'canceled')
ORDER BY f.scheduled_time DESC;
"""
pd.read_sql(query_9, conn)

,flight_number,aircraft_registration,origin_airport,destination_airport,scheduled_time
0,6E 6466,None,None,None,2026-01-02 17:20Z
1,AF 660,None,Dubai,None,2026-01-02 16:10Z
2,6E 6423,None,None,None,2026-01-02 15:00Z
3,6E 757,None,None,Kolkata Netaji Subhash Chandra Bose,2026-01-02 14:20Z
4,S5 151,None,Hyderabad Rajiv Gandhi,None,2026-01-02 14:10Z
5,TR 637,None,None,None,2026-01-02 12:40Z
6,6E 6465,None,None,None,2026-01-02 12:30Z
7,JL 193,None,None,None,2026-01-02 11:40Z
8,6E 926,None,None,None,2026-01-02 11:35Z
9,6E 757,None,Bangalore Bengaluru,None,2026-01-02 11:30Z


In [ ]:
query_10 = """
SELECT
    o.city AS origin_city,
    d.city AS destination_city,
    COUNT(DISTINCT ac.model) AS aircraft_models
FROM flights f
LEFT JOIN aircraft ac
    ON f.aircraft_registration = ac.registration
JOIN airport o
    ON f.origin_iata = o.iata_code
JOIN airport d
    ON f.destination_iata = d.iata_code
GROUP BY o.city, d.city
HAVING COUNT(DISTINCT ac.model) > 2;
"""
pd.read_sql(query_10, conn)

,origin_city,destination_city,aircraft_models


In [19]:
query_11 = """
SELECT
    a.name AS destination_airport,
    ROUND(
        100.0 * SUM(CASE WHEN f.status = 'Delayed' THEN 1 ELSE 0 END)
        / COUNT(*),
        2
    ) AS delayed_percentage
FROM flights f
JOIN airport a
    ON f.destination_iata = a.iata_code
WHERE f.flight_type = 'arrival'
GROUP BY a.name
ORDER BY delayed_percentage DESC;
"""
pd.read_sql(query_11, conn)


,destination_airport,delayed_percentage
0,Hyderabad Rajiv Gandhi,15.00
1,Dubai,8.70
2,New York John F Kennedy,6.67
3,Mumbai Chhatrapati Shivaji,5.56
4,Chennai,5.56
5,New Delhi Indira Gandhi,4.76
6,Tokyo,0.00
7,Sydney Kingsford Smith,0.00
8,Singapore Changi,0.00
9,Paris Charles de Gaulle,0.00
